In [1]:
# here we will implement a simple pretrained model to predict a emoji 
# given a sentence. This is a starting level implementation.
# importing required libraries
import numpy as np
import pandas as pd
import emoji as em
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# reading the data via pandas
data = pd.read_csv('./emoji_data.csv', header=None) # as there is no header in data
# data[1].value_counts()

In [3]:
# as we can see the data have text and numbers. The numbers represents the 
# category of the emoji. TO use them, we need emoji librari to create mapping
# dictionary
# the emoji library has function emojize to print any emoji, such as:
em.emojize(':thumbs_up:')

'👍'

In [4]:
# lets map the integers values in our data to emojies
# lets create a dictionary 
emoji_dict = {
    0:':red_heart:',
    1: ':baseball:',
    2: ':grinning_face_with_big_eyes:',
    3: ':disappointed_face:',
    4: ':fork_and_knife_with_plate:'
}
emoji_dict

{0: ':red_heart:',
 1: ':baseball:',
 2: ':grinning_face_with_big_eyes:',
 3: ':disappointed_face:',
 4: ':fork_and_knife_with_plate:'}

In [5]:
# defing a function to where we will pass the label (key) and it will
# return us the emoji
def label_to_emoji(label):
    return em.emojize(emoji_dict[label])

label_to_emoji(0), label_to_emoji(1), label_to_emoji(3)

('❤️', '⚾', '😞')

In [6]:
# now lets separate the data from labels - here we will use 
# x -  as data and y - as labels
x = data[0].values
y = data[1].values
# x, y

In [7]:
# it is time to use word embedding - I will be using pre-trained word embedding
# glove, introduced by stanford researchers
# please follow this link to download: http://nlp.stanford.edu/data/glove.6B.zip
# once it is downloaded: extract it and it will have four files
# each file represents the embedding with respect to their dimensions
# here, 100d file will be used - lets open it using open()
file = open('./glove.6B/glove.6B.100d.txt', encoding='utf8')
content = file.readlines()
file.close()

In [8]:
# content[:50]
# here we can see that against each word there is word vector of 100 dimension

In [9]:
# we are going to create a dictionary that will map the word to their 
# corresponding vector
embedding = {}

# lets iterate through each line;
for line in content:
    line = line.split()
    
    # we are going to map the first word against the remaining vector
    embedding[line[0]] = np.array(line[1:], dtype=float)


In [10]:
len(embedding.keys()) # here we can see that we have 400000 words

400000

In [11]:
# next thing we have to preprocess the data - converting the data into input 
# tokens and convert y into one-hot vector
# for tokening the input text, a tokenizer function from keras will be used
# it will convert each specific word from text into a number and number is 
# assigned to a order in which that word is appearing in the dictionary
# lets initiate the tokenizer
tokenizer = Tokenizer()
tokenizer

In [12]:
# lets fit this onto our dataset
tokenizer.fit_on_texts(x)

In [13]:
# lets see the output of above line
word_to_index = tokenizer.word_index
len(word_to_index)

312

In [14]:
# next all words will be converted into a list of tokens
x_tokens = tokenizer.texts_to_sequences(x)
# x_tokens

In [15]:
# here we cann see that all tokens have arbitrary length -  this cannot be used 
# directly in the model. we need a fixed length input
# we will be using padding to transfer it into a fixed length vector - the fixed
# length will be the largest vector of this dataset
maxlen = 0
for data in x_tokens:
    maxlen = max(maxlen, len(data))


In [16]:
maxlen # the max length of our sentence here is:

10

In [17]:
# lets padd the x_tokens and called it training data
x_train = pad_sequences(x_tokens, maxlen=maxlen, padding='post', truncating='post')
# x_train[:40]

In [18]:
# the x_data is ready to be used for training
# next convert the y into one-hot encoding
# as our data contain some values which we  don't need, we need to clean the
# data
y_ = [(lambda x: x.split()[0][0]) (x) for x in y.tolist()]
y = np.array(y_, dtype=object)
# y

In [19]:
# lets convert it into one-hot, for that, we will be using a builtin 
# function to_categorical
y_train = to_categorical(y)
# y_train[:10]

# Model implementation

In [20]:
# here we are going to use pretrained word embeddings - for that we will pass 
# matrix to weights parameters in the models. Lets create that matrix
# here that matrix will have all the words embedding vectors of all the words in
# our dataset
# here the rows will be the words and the columns will be the vectors
embed_size = 100 # as we are using 100 dimension vector

# lets initialize the matrix will zero values
embedding_matrix = np.zeros((len(word_to_index)+1, embed_size))

# lets iterate over all the words in our embedding dict
for word, index in word_to_index.items():
    embed_vector = embedding[word]
    embedding_matrix[index] = embed_vector


In [21]:
# embedding_matrix

In [22]:
# lets create the model now
model = Sequential([
    Embedding(input_dim=len(word_to_index)+1,
             output_dim=embed_size,
             input_length=maxlen,
              weights=[embedding_matrix],
              trainable=False
             ),
    # next we can specify simple LSTM or RNN layer - first LSTM
    LSTM(units=16, return_sequences=True), # return_sequence will make sure to 
                                            # return the value at each sequence
     LSTM(units=10, return_sequences=True),
    LSTM(units=4), # we do not have to specify the return_sequence at last layer
    Dense(units=5, activation='softmax'),
    
])

model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 100)           31300     
                                                                 
 lstm (LSTM)                 (None, 10, 16)            7488      
                                                                 
 lstm_1 (LSTM)               (None, 10, 10)            1080      
                                                                 
 lstm_2 (LSTM)               (None, 4)                 240       
                                                                 
 dense (Dense)               (None, 5)                 25        
                                                                 
Total params: 40,133
Trainable params: 8,833
Non-trainable params: 31,300
_________________________________________________________________


In [24]:
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
6/6 [==============================] - 9s 23ms/step - loss: 1.6063 - accuracy: 0.2951
Epoch 2/50
6/6 [==============================] - 0s 20ms/step - loss: 1.5938 - accuracy: 0.3552
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 1.5842 - accuracy: 0.3607
Epoch 4/50
6/6 [==============================] - 0s 20ms/step - loss: 1.5752 - accuracy: 0.3060
Epoch 5/50
6/6 [==============================] - 0s 21ms/step - loss: 1.5652 - accuracy: 0.2896
Epoch 6/50
6/6 [==============================] - 0s 20ms/step - loss: 1.5536 - accuracy: 0.2896
Epoch 7/50
6/6 [==============================] - 0s 22ms/step - loss: 1.5393 - accuracy: 0.3115
Epoch 8/50
6/6 [==============================] - 0s 20ms/step - loss: 1.5180 - accuracy: 0.3279
Epoch 9/50
6/6 [==============================] - 0s 22ms/step - loss: 1.4951 - accuracy: 0.3443
Epoch 10/50
6/6 [==============================] - 0s 23ms/step - loss: 1.4690 - accuracy: 0.3770
Epoch 11/50
6/6 [============

In [25]:
# lets test the model - I am going to write the sentences to test
test = ['i feel good', 'i feel very bad', 
        'lets eat dinner']
# lets convert the words into padded tokenize sequences
test_sequ = tokenizer.texts_to_sequences(test)
x_test = pad_sequences(test_sequ, maxlen=maxlen, padding='post', 
                      truncating='post')

In [26]:
# lets make the predictions
y_predict = model.predict(x_test)
y_predict

1/1 [==============================] - 2s 2s/step


array([[0.10998087, 0.23925649, 0.45704293, 0.09805621, 0.09566353],
       [0.01549632, 0.0602515 , 0.4272157 , 0.4523882 , 0.04464825],
       [0.20270593, 0.06507625, 0.08274296, 0.08793829, 0.5615365 ]],
      dtype=float32)

In [27]:
# # lets get the index where max value is present
y_pred = np.argmax(y_predict, axis=1)
y_pred

array([2, 3, 4], dtype=int64)

In [28]:
# we have our tokens, lets see the emojies now
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

i feel good 😃
i feel very bad 😞
lets eat dinner 🍽️


In [29]:
# this was just an initial level implementation - model is making mistakes
# we can use large dataset to improve the results